# Deduplicating Common-Crawl for improve LLM Training Quality. 

In this notebook, we will cover how to perform the minhash deduplication algorithm on the super-massive common crawl dataset.

Common crawl is an index of public webpages famously used to train GPT-3. LLMs require massive amounts of data 


Common Crawl is a super-massive web dataset dating 

In [26]:
import daft
from daft import col
from daft.io import IOConfig, S3Config
import os
from dotenv import load_dotenv

load_dotenv()

s3_config = S3Config(
    region_name="us-east-1",
    requester_pays=True,
    key_id=os.environ["AWS_ACCESS_KEY_ID"],
    access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
    anonymous=False,
)

IO_CONFIG = IOConfig(s3=s3_config)
daft.set_planning_config(default_io_config=IO_CONFIG)

DaftContext(_ctx=<builtins.PyDaftContext object at 0x11c369e70>)

In [35]:
df_warc = daft.read_warc("s3://commoncrawl/crawl-data/CC-MAIN-2018-17/segments/1524125937193.1/warc/*.warc.gz").limit(10)
df_warc.show()

WARC-Record-IDUtf8,WARC-TypeUtf8,"WARC-DateTimestamp(Nanoseconds, Some(""Etc/UTC""))",Content-LengthInt64,WARC-Identified-Payload-TypeUtf8,warc_contentBinary,warc_headersUtf8
bc13aa27-5dbc-4f25-aeab-c802a3ab7ef3,warcinfo,2018-04-20 08:14:00 UTC,324,None,"b""robots: classic\r\nhostname: ip-10""...","{""Content-Type"":""application/warc-fields"",""WARC-Filename"":""CC-MAIN-20180420081400-20180420101400-00000.warc.gz""}"
c23396b2-da8f-44ad-8e28-c72d2d152129,request,2018-04-20 08:32:21 UTC,286,None,"b""GET /news-ed-eventi/news-seat HTTP""...","{""Content-Type"":""application/http; msgtype=request"",""WARC-IP-Address"":""89.118.107.132"",""WARC-Target-URI"":""http://00064.dealerseat.com/news-ed-eventi/news-seat"",""WARC-Warcinfo-ID"":""<urn:uuid:bc13aa27-5dbc-4f25-aeab-c802a3ab7ef3>""}"
295bd657-a3ef-4c28-8027-ec136a026ceb,response,2018-04-20 08:32:21 UTC,385901,text/html,"b""HTTP/1.1 200 Apple\r\nServer: ngin""...","{""Content-Type"":""application/http; msgtype=response"",""WARC-Block-Digest"":""sha1:POWDQGDGUYEFWFCP5TLO5VL67EY7BH6K"",""WARC-Concurrent-To"":""<urn:uuid:c23396b2-da8f-44ad-8e28-c72d2d152129>"",""WARC-IP-Address"":""89.118.107.132"",""WARC-Payload-Digest"":""sha1:ACTPVYHET7H243TRUCUNM2FUI7PIY6MD"",""WARC-Target-URI"":""http://00064.dealerseat.com/news-ed-eventi/news-seat"",""WARC-Truncated"":""length"",""WARC-Warcinfo-ID"":""<urn:uuid:bc13aa27-5dbc-4f25-aeab-c802a3ab7ef3>""}"
777d0888-fecc-4a6b-9ed7-167c8611cf0c,metadata,2018-04-20 08:32:21 UTC,21,None,"b""fetchTimeMs: 1307\r\n\r\n""","{""Content-Type"":""application/warc-fields"",""WARC-Concurrent-To"":""<urn:uuid:295bd657-a3ef-4c28-8027-ec136a026ceb>"",""WARC-Target-URI"":""http://00064.dealerseat.com/news-ed-eventi/news-seat"",""WARC-Warcinfo-ID"":""<urn:uuid:bc13aa27-5dbc-4f25-aeab-c802a3ab7ef3>""}"
c5a31119-976e-4c4e-939a-9dbfb040e3de,request,2018-04-20 08:32:40 UTC,269,None,"b""GET /kouchi/goiken/ HTTP/1.0\r\nHo""...","{""Content-Type"":""application/http; msgtype=request"",""WARC-IP-Address"":""218.224.226.141"",""WARC-Target-URI"":""http://00monthly.com/kouchi/goiken/"",""WARC-Warcinfo-ID"":""<urn:uuid:bc13aa27-5dbc-4f25-aeab-c802a3ab7ef3>""}"
5e643323-ea8e-4a4a-a1c3-6fc61791dd08,response,2018-04-20 08:32:40 UTC,10835,application/xhtml+xml,"b""HTTP/1.1 200 OK\r\nDate: Fri, 20 A""...","{""Content-Type"":""application/http; msgtype=response"",""WARC-Block-Digest"":""sha1:FD5A7ID6JPOT3XUAMFX3XNWDD2APIVBE"",""WARC-Concurrent-To"":""<urn:uuid:c5a31119-976e-4c4e-939a-9dbfb040e3de>"",""WARC-IP-Address"":""218.224.226.141"",""WARC-Payload-Digest"":""sha1:WEHISROIBSQ5IM2BKJDKJ3FRG4GSDZ5Y"",""WARC-Target-URI"":""http://00monthly.com/kouchi/goiken/"",""WARC-Truncated"":""length"",""WARC-Warcinfo-ID"":""<urn:uuid:bc13aa27-5dbc-4f25-aeab-c802a3ab7ef3>""}"
16de518d-0ae1-4f61-8cfb-97c11b7e86c5,metadata,2018-04-20 08:32:40 UTC,20,None,"b""fetchTimeMs: 620\r\n\r\n""","{""Content-Type"":""application/warc-fields"",""WARC-Concurrent-To"":""<urn:uuid:5e643323-ea8e-4a4a-a1c3-6fc61791dd08>"",""WARC-Target-URI"":""http://00monthly.com/kouchi/goiken/"",""WARC-Warcinfo-ID"":""<urn:uuid:bc13aa27-5dbc-4f25-aeab-c802a3ab7ef3>""}"
13cd06fc-beb5-4285-a903-c1fa531b3f85,request,2018-04-20 08:35:13 UTC,236,None,"b""GET /pissysexy21-video HTTP/1.0\r\n""...","{""Content-Type"":""application/http; msgtype=request"",""WARC-IP-Address"":""144.217.113.16"",""WARC-Target-URI"":""http://00secretsweety.ozforex.info/pissysexy21-video"",""WARC-Warcinfo-ID"":""<urn:uuid:bc13aa27-5dbc-4f25-aeab-c802a3ab7ef3>""}"


Here is a funny friction point... how do I read warc after loading the the paths...

In [30]:
# Lets inspect the content of a single warc file
content_example= df_warc.to_pydict()["warc_content"][0]
print(content_example)

/Users/everett-founder/git/ugh/daft-minhash-dedupe/.venv/lib/python3.11/site-packages/daft/dashboard/__init__.py:91: UserWarning: Failed to broadcast metrics over http://127.0.0.1:3239/api/queries: HTTP Error 400: Bad Request
  warnings.warn(f"Failed to broadcast metrics over {url}: {e}")


b'robots: classic\r\nhostname: ip-10-158-89-8.ec2.internal\r\nsoftware: Nutch 1.6 (CC)\r\nisPartOf: CC-MAIN-2018-17\r\noperator: Common Crawl Admin\r\ndescription: Wide crawl of the web for April 2018\r\npublisher: Common Crawl\r\nformat: WARC File Format 1.0\r\nconformsTo: http://bibnum.bnf.fr/WARC/WARC_ISO_28500_version1_latestdraft.pdf\r\n'


In [36]:
# Decode WARC content to text and add a doc_id
from daft import functions as F

# Note: `warc_content` is binary; decode to UTF-8 text

df = df_warc.with_column(
    "content_text",
    col("warc_content").try_decode("utf-8").str.split('\r\n')
)

# Add a stable row id for later grouping
df = df.with_column("doc_id", F.monotonically_increasing_id())

df.select("doc_id", "content_text").show(3)



The Minhash Deduplication algorithm



In [ ]:
# pip install selectolax
from selectolax.parser import HTMLParser
import re
import hashlib

@daft.udf(return_dtype=daft.DataType.list(daft.DataType.string()))
def extract_blocks(html: str) -> list[str]:
    tree = HTMLParser(html)
    for n in tree.css("script,style,noscript"): n.decompose()
    blocks = []
    for node in tree.css("article, main, p, h1, h2, h3, li"):
        txt = node.text(separator=" ", strip=True)
        if not txt: 
            continue
        # link density filter
        links = len(node.css("a"))
        letters = sum(ch.isalpha() for ch in txt)
        if links > 3 and links > letters / 20:
            continue
        if letters < 40:
            continue
        blocks.append(txt)
    return blocks

def block_shingles(html: str) -> list[int]:
    out = set()
    for blk in extract_blocks(html):
        s = blk.lower()
        s = re.sub(r"[^a-z0\s]+", " ", s)
        s = re.sub(r"\s+", " ", s).strip()
        if len(s) < 40: 
            continue
        h = hashlib.sha1(s.encode()).digest()[:8]
        out.add(int.from_bytes(h, "big"))
    return list(out)


In [ ]:
K = 5          # shingle size
M = 64         # number of hash permutations
R = 8          # rows per band
B = M // R     # bands

## Preprocessing
with 64 bit int shingle 

In [29]:
# K-word shingles 

df_prep = df_warc.with_column("word_tokens", 
    col("content_string").str.normalize(
        remove_punct=True, 
        lowercase=True, 
        nfd_unicode=True, 
        white_space=True
    ).str.replace(r"\d","0",True
    ).str.split(" ") 
)

df_prep.show(3)




DaftCoreException: DaftError::FieldNotFound Column col(content_string) not found.

In [ ]:
# Tokenize text using Daft string expressions
K = 5

# normalize, collapse whitespace, replace digits, split into tokens

df_tokens = df.with_column(
    "tokens",
    col("content_text").str.normalize(
        remove_punct=False,
        lowercase=True,
        nfd_unicode=True,
        white_space=True,
    ).str.replace(r"\d", "0", regex=True)
     .str.split(" ")
)

df_tokens.select("doc_id", "tokens").show(3)



In [22]:
# UDF: generate k-word shingles as 64-bit integers
import re, hashlib
from daft import lit

def k_shingles(tokens: list[str], k: int = 5) -> list[int]:
    if not tokens:
        return []
    norm = [t for t in tokens if t]
    if len(norm) < k:
        return []
    seen = set()
    out: list[int] = []
    for i in range(len(norm) - k + 1):
        gram = " ".join(norm[i:i+k])
        h = hashlib.sha1(gram.encode()).digest()[:8]
        v = int.from_bytes(h, "big")
        if v not in seen:
            seen.add(v)
            out.append(v)
    return out

udf_kshingles = daft.udf(k_shingles, return_dtype=daft.DataType.list(daft.DataType.int64()))

# apply UDF

df_shingles = df_tokens.with_column(
    "shingles",
    udf_kshingles(col("tokens"), lit(K))
)

df_shingles.select("doc_id", "shingles").show(3)



TypeError: udf() takes 0 positional arguments but 1 positional argument (and 1 keyword-only argument) were given

In [ ]:
# Minhash signature computation
M = 64

import random
random.seed(42)

# Pre-generate M (a, b) pairs for hash functions over 64-bit space
A = [random.randrange(1, 2**61-1) for _ in range(M)]
B = [random.randrange(0, 2**61-1) for _ in range(M)]
P = (1 << 61) - 1  # a large prime (2^61-1 is prime)
MASK64 = (1 << 64) - 1

from typing import List

def minhash_signature(shingles: List[int], A: List[int], B: List[int], P: int) -> List[int]:
    if not shingles:
        return [MASK64] * len(A)
    sig = []
    for a, b in zip(A, B):
        m = MASK64
        for s in shingles:
            # universal hash on 64-bit shingles, then fold to 64-bit
            h = ((a * s + b) % P) & MASK64
            if h < m:
                m = h
        sig.append(m)
    return sig

udf_minhash = daft.udf(
    minhash_signature,
    return_dtype=daft.DataType.list(daft.DataType.int64())
)

df_sig = df_shingles.with_column(
    "signature",
    udf_minhash(col("shingles"), lit(A), lit(B), lit(P))
)

df_sig.select("doc_id", col("signature").list.length().alias("M"), "signature").show(3)



In [ ]:
# LSH banding and candidate generation
R = 8
BANDS = M // R

from typing import List, Tuple

def band_signature(signature: List[int], rows: int, bands: int) -> List[Tuple[int, int]]:
    # returns list of (band_index, band_hash)
    if not signature:
        return []
    out: List[Tuple[int, int]] = []
    for b in range(bands):
        start = b * rows
        end = start + rows
        if end > len(signature):
            break
        # hash the slice into a 64-bit bucket
        h = 1469598103934665603  # FNV offset
        for v in signature[start:end]:
            h ^= (v & 0xFFFFFFFFFFFFFFFF)
            h = (h * 1099511628211) & 0xFFFFFFFFFFFFFFFF
        out.append((b, h))
    return out

udf_band = daft.udf(
    band_signature,
    return_dtype=daft.DataType.list(daft.DataType.struct({"band": daft.DataType.int64(), "key": daft.DataType.int64()}))
)

# attach bands

df_bands = df_sig.with_column(
    "bands",
    udf_band(col("signature"), lit(R), lit(BANDS))
)

# explode to one row per (band, key)

df_pairs = df_bands.select("doc_id", col("bands").unnest().alias("pair")).select(
    "doc_id",
    col("pair").struct.get("band").alias("band"),
    col("pair").struct.get("key").alias("key"),
)

# group by (band, key) to collect candidate doc_ids

df_cands = df_pairs.groupby("band", "key").agg(col("doc_id").agg_list().alias("doc_ids"))

df_cands.show(5)



In [ ]:
# Aggregate clusters and preview

# Keep only buckets with at least 2 docs

df_clusters = df_cands.where(col("doc_ids").list.length() >= 2)

# Optional: compute a representative for each cluster (smallest doc_id)

df_clusters = df_clusters.with_column(
    "rep", col("doc_ids").list.sort().list.get(0)
)

# Explode to doc-level mapping
df_cluster_docs = df_clusters.select(
    "band", "key", "rep", col("doc_ids").unnest().alias("doc_id")
)

# Join back to optional metadata/text if desired
preview = df_cluster_docs.join(df.select("doc_id", "content_text"), on=["doc_id"])\
    .select("rep", "doc_id", "band", "key", "content_text")

preview.show(10)

